In [18]:
import pickle
import json
import os
from gensim.models import Word2Vec
import gensim
import numpy as np

In [3]:
data = pickle.load(open("./unknow_data/relation_dict.pkl", 'rb'))


{'/people/deceased_person/place_of_death': 4,
 '/people/person/place_lived': 3,
 '/people/person/place_of_birth': 2,
 'NA': 0,
 'per:origin': 1}

In [4]:
wiki_relation_map = json.load(open("WikipediaWikidataDistantSupervisionAnnotations.v1.0/en.json", 'r'))

### Word to Vector model.
using gensim to implement.
save in 'Preprocess_model_pkl/'

In [42]:
class MySentences(object):
    def __init__(self, filename):
        self.filename = filename
 
    def __iter__(self):
#         for fname in os.listdir(self.dirname):
#             for line in open(os.path.join(self.dirname, fname)):
#                 yield line.split()
         for line in open(self.filename):
            temp_sentence = line.split('\t')[-1]
            yield temp_sentence.split()


In [35]:
def GetEntityPos(entity1, entity2, sentence):
    pos_1 = 0
    pos_2 = 0
    for i in range(len(sentence)):
        if sentence[i] == entity1:
            pos_1 = i
        if sentence[i] == entity2:
            pos_2 = i
    pos_first = min(pos_1, pos_2)
    pos_second = pos_1 + pos_2 - pos_first
    return [pos_first, pos_second]

In [60]:
def LoadFile(filename, max_length, vec_size, pos_size, word_2_vec_model, relation_set):
    data_embedding = None
    relation_label = []
    print("Process file:",filename)
    count = 0
    for line in open(filename):
        if (count%1000) == 0:
            print("Now the count is: ",count)
        count += 1
        split_sentce = line.split('\t')
        relation_id = split_sentce[2]
        flag = 0
        for (key, relation) in relation_set.items():
            if relation[1] == relation_id:
                relation_id = relation[0]
                flag = 1
                break
        if flag == 0:
            continue
        relation_label.append(relation_id)
        entity_1 = split_sentce[0]
        entity_2 = split_sentce[1]
        words = split_sentce[-1].split()
        word_embedding = np.zeros((max_length, vec_size))
        pos_embedding = np.zeros((max_length, pos_size))
        (entity_pos_1, entity_pos_2) = GetEntityPos(entity_1, entity_2, words)
        pos_embedding[entity_pos_1] = np.ones(pos_size)
        pos_embedding[entity_pos_2] = np.ones(pos_size)
        for i in range(len(words)):
            word_embedding[i] = word_2_vec_model[words[i]]
        input_embedding = np.concatenate((word_embedding, pos_embedding), axis = 1)
        input_embedding = input_embedding.reshape(1, max_length, -1)
        if data_embedding is None:
            data_embedding = input_embedding
        else:
            data_embedding = np.concatenate((data_embedding, input_embedding), axis = 0)
    return (data_embedding,relation_label)

In [62]:
is_training = False

wiki_data_path = './data/wiki_data/'
files = os.listdir(wiki_data_path)
wiki_file = './data/wiki_data/filterer-held-out.txt'
if is_training:
    
    # temp_file = open(wiki_file, 'r')
    # temp_file_lines = [line for line in temp_file.readlines()]
    sentences = MySentences(wiki_file)
    model = gensim.models.Word2Vec(sentences, min_count=1, size = 50)
    model.save('Preprocess_model_pkl/wiki_vec_model.pkl')
else:
    model = gensim.models.Word2Vec.load('Preprocess_model_pkl/wiki_vec_model.pkl')
relation_set = pickle.load(open('data/common_relations.pkl', 'rb'))


In [63]:
count = 0
for line in open(wiki_file):
    count += 1
print(count)

324704


In [ ]:
data = LoadFile(wiki_file, 100, vec_size=50, pos_size=5, word_2_vec_model=model, relation_set=relation_set)

Process file: ./data/wiki_data/filterer-held-out.txt
Now the count is:  0


/Users/sun/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Now the count is:  1000
Now the count is:  2000
Now the count is:  3000
Now the count is:  4000
Now the count is:  5000
Now the count is:  6000
Now the count is:  7000
Now the count is:  8000
Now the count is:  9000
Now the count is:  10000
Now the count is:  11000
Now the count is:  12000
Now the count is:  13000
Now the count is:  14000
Now the count is:  15000
Now the count is:  16000
Now the count is:  17000
Now the count is:  18000
Now the count is:  19000
Now the count is:  20000
Now the count is:  21000
Now the count is:  22000
Now the count is:  23000
Now the count is:  24000
Now the count is:  25000
Now the count is:  26000
Now the count is:  27000
Now the count is:  28000
Now the count is:  29000
Now the count is:  30000
Now the count is:  31000
Now the count is:  32000
Now the count is:  33000
Now the count is:  34000
Now the count is:  35000
Now the count is:  36000
Now the count is:  37000
Now the count is:  38000
Now the count is:  39000
Now the count is:  40000
Now the c

In [2]:
import numpy as np
data = np.load('data/processed_data/wiki_held-out_data.npy')

In [3]:
data.shape

(324704, 100, 55)